# Contents
[Clean legacy users from SQL database](#cleaning-data-from-the-sql-database-from-aws)  
[Clean card details from S3 bucket PDF data](#data-cleaning-of-card-details-from-the-s3-bucket)  
[Code to GET API data](#checking-code-to-get-api-data)  
[Clean store details fromAPI data](#reviewing-and-cleaning-the-api-data)  
[Clean product data from S3 bucket CSV data](#extracting-s3-object-using-boto3)  
[Clean order tables from AWS RDS](#tidy-order-table)  
[Cleaning sales date from AWS S3](#extracting-sales_date-from-s3)


### Cleaning data from the SQL database from AWS

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

with open('db_creds.yaml','r') as file:
            credentials = yaml.safe_load(file)

DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = credentials['RDS_HOST']
USER = credentials['RDS_USER']
PASSWORD = credentials['RDS_PASSWORD']
DATABASE = credentials['RDS_DATABASE']
PORT = credentials['RDS_PORT']
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

with engine.connect() as conn:
    db_table = pd.read_sql_table('legacy_users', conn, chunksize= None)

In [75]:
db_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [76]:
db_table.loc[db_table['user_uuid'] == '9476f17e-5d6a-4117-874d-9cdb38ca1fa6']

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


In [4]:
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
import pandas as pd
import numpy as np

db = DatabaseConnector()
table = DataExtractor()


In [5]:
df = table.read_rds_table(db, 'legacy_users')
df = df.drop('index',axis=1)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB


In [10]:
import pandas as pd
df.to_csv("review2.csv")

In [66]:
df.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [67]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB
None


In [11]:
df.loc[df['user_uuid'] == '0423a395-a04d-4e4a-bd0f-d237cbd5a295']

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


In [9]:
target_string = '0423'
matched_rows = df[df['user_uuid'].str.contains(target_string)]
print(matched_rows)

      first_name last_name date_of_birth                      company  \
9023      Teresa    Little    2005-10-04  Watson, Carter and Thompson   
14145    Anthony     Ahmed    1948-04-07               Richardson LLC   

                   email_address                               address  \
9023   lawrencechan@mccarthy.com   7 Sandra field\nPaigefurt\nNP4P 6JP   
14145     opowell@gill-smith.com  65 Hughes curve\nWest Terry\nCW5 2DF   

              country country_code     phone_number   join_date  \
9023   United Kingdom           GB  (0115) 496 0307  2022-11-08   
14145  United Kingdom           GB     +44292018682  2009-04-09   

                                  user_uuid  
9023   5ab6d2f3-710c-439c-aac7-e07309804239  
14145  f6f8e03d-ebbd-44b1-adf2-30cd830423f1  


In [68]:
# First process the the date columns 
# Converting any date columns into the date format
import pandas as pd
from dateutil.parser import parse
for col in df.columns:  
    if 'date' in col:
        def try_parsing_date(text):
            try:
                return parse(text)
            except Exception:
                return pd.NaT
        
        df[col] = df[col].apply(try_parsing_date)
        df[col] = pd.to_datetime(df[col], errors='coerce')



In [69]:
from datetime import datetime
# Calculate the current date
current_date = datetime.now()
# Subset the data to show individuals older than 70 years
subset_older_than_70 = df[current_date.year - df['date_of_birth'].dt.year > 80]
print(subset_older_than_70)

      first_name last_name date_of_birth                      company  \
1            Guy     Allen    1940-12-01                      Fox Ltd   
21          Jörg  Hoffmann    1940-11-04                        Höfig   
66        Olivia  Williams    1940-10-23  Sanderson, Singh and Wilson   
77      Klaus-D.      Gute    1942-06-21        Conradi GmbH & Co. KG   
84       Dominic    Potter    1940-01-19                 Powell-Carey   
...          ...       ...           ...                          ...   
15274       Joan      Reed    1942-10-24                    Frost PLC   
15289    Stephen     Davis    1940-08-30    Graham, Fisher and Gibson   
15292    Stephen      Ward    1940-07-22                  Smith-Brown   
15308    Stephen    Hudson    1940-09-13                   Harris PLC   
15317    Stephen  Losekann    1940-10-09                      Rosenow   

                        email_address  \
1        rhodesclifford@henderson.com   
21             zgehringer@beckmann.de   


In [70]:
#Removing NA rows of data
#Further inspection show all NA rows in date_of_birth include non-null rows but are randomised.
nat_indexes = df.index[df['date_of_birth'].isna()].tolist()
print(nat_indexes)
na_list = df.iloc[nat_indexes]
print(na_list)

[752, 866, 1022, 1046, 1805, 2103, 2437, 2739, 2764, 2995, 3536, 4984, 5306, 5307, 6420, 6920, 7737, 8386, 9013, 10013, 10211, 10224, 10360, 10988, 11366, 11443, 11598, 11761, 11864, 12092, 12177, 12584, 13111, 13855, 14101, 14499]
       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84         

In [71]:
# Double checking if the parse and to_datetime function worked on dates that had string e.g. October 
# Anti join show na_list removes the randomised and unparsable dates and leaves the good ones in
result = df[df['date_of_birth'].astype(str).str.contains('[A-Za-z]', regex=True, na=False)]
anti_join_df = result.merge(na_list, left_index=True, right_index=True, how='left', indicator=True)
anti_join_result = anti_join_df[anti_join_df['_merge'] == 'left_only']
print(result)
print(anti_join_result)

       first_name   last_name date_of_birth     company email_address  \
752    PYCLKLLC7I  W350SCUD6R           NaT  R7IZUNSQX0    3Q791B3VIY   
866          NULL        NULL           NaT        NULL          NULL   
1022         NULL        NULL           NaT        NULL          NULL   
1046   GI4C78KWH0  UTB5PPYFG8           NaT  CA1XGS8GZW    7HSZB429UK   
1805         NULL        NULL           NaT        NULL          NULL   
2103         NULL        NULL           NaT        NULL          NULL   
2437         NULL        NULL           NaT        NULL          NULL   
2739         NULL        NULL           NaT        NULL          NULL   
2764         NULL        NULL           NaT        NULL          NULL   
2995   DPAJNJL6PR  B8ZGN8ZJ84           NaT  0YJ2FRMDB4    O5Q6D7FDAF   
3536   E5139W1FJ3  X88YLZX0JX           NaT  648S19YUFF    BYU3R2LQF8   
4984         NULL        NULL           NaT        NULL          NULL   
5306   XKDVQD7BH2  4YSEX8AY1Z           NaT  GKJZ58

In [72]:
# So to conclude, all nulls in date of birth removed, this also removes randomised records of data
df = df.drop(nat_indexes)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   15284 non-null  object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.4+ MB


In [74]:
# This shows no NA values are left. Following scripts would be to look for and handle incorrect data
na_counts = df.isna().sum()
print(na_counts)

first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


In [75]:
# Next fix the phone numbers
# The below processing of US, UK and Germany has been combined into this single cell
us_indexes = df[df['country'] == "United States"].index
other_indexes = df[df['country'] != "United States"].index

df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])', '', regex=True)
df.loc[us_indexes, 'phone_number'] = df.loc[us_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]', '', regex=True)
df.loc[other_indexes, 'phone_number'] = df.loc[other_indexes, 'phone_number'].apply(lambda x: np.nan if len(str(x)) != 11 else x)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [77]:
#Review country_code
df['country_code'].value_counts()

country_code
GB     9365
DE     4708
US     1205
GGB       6
Name: count, dtype: int64

In [78]:
df.loc[df['country_code'] == "GGB", 'country_code'] = "GB"

In [79]:
# Next country - all good
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [80]:
# assumption that the address column is correct, could separate postcode and cross-reference on google maps
print(df['address'][0])

Zimmerstr. 1/0
59015 Gießen


In [81]:
# Remove the new line and replace with comma
df['address'] = df['address'].str.replace('\n', ',')
print(df['address'])

0                             Zimmerstr. 1/0,59015 Gießen
1          Studio 22a,Lynne terrace,McCarthymouth,TF0 9GH
2                        92 Ann drive,Joanborough,SK0 6LR
3                    19 Robinson meadow,New Tracy,W22 2QG
4                     3 White pass,Hunterborough,NN96 4UE
                               ...                       
15315         Studio 41I,Jones lodge,Oliviaborough,E8 3DU
15316                 530 Young parkway,Millsfurt,L4G 7NX
15317    Viviane-Fritsch-Straße 3/5,15064 Bad Liebenwerda
15318     660 Ross Falls Suite 357,Anthonymouth, MA 09610
15319         Studio 4,Hancock road,Phillipsview,ST1X 3XB
Name: address, Length: 15284, dtype: object


In [82]:
#All email addresses follow a consistent pattern
df['email_address'].str.contains('@.*\.', regex=True).value_counts()

email_address
True    15284
Name: count, dtype: int64

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  object        
 7   country_code   15284 non-null  object        
 8   phone_number   9089 non-null   object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), object(9)
memory usage: 1.9+ MB


In [84]:

print(df['first_name'].str.contains(r'\d', regex=True).value_counts())
print(df['last_name'].str.contains(r'\d', regex=True).value_counts())
print(df['company'].str.contains(r'\d', regex=True).value_counts())

first_name
False    15284
Name: count, dtype: int64
last_name
False    15284
Name: count, dtype: int64
company
False    15284
Name: count, dtype: int64


In [85]:
df['first_name'].value_counts()

first_name
Michael       57
Amanda        52
Peter         51
Jennifer      50
Andrew        50
              ..
Kristopher     1
Guido          1
Darlene        1
Adem           1
Athanasios     1
Name: count, Length: 2162, dtype: int64

In [86]:
#Below was the analysis of numbers from different countries

In [87]:
# Next fix the phone numbers
df['country'].value_counts()

country
United Kingdom    9371
Germany           4708
United States     1205
Name: count, dtype: int64

In [88]:
#

In [89]:
#10 digits
us = df[df['country'] == 'United States']
print(us)
print(us.info())

      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5              mwilliams@nichols.org   
9                 t

In [90]:
# For the USA, typically follow XXX-XXX-XXXX pattern
us.loc[:,'phone_number'] = us['phone_number'].str.replace(r'(\+\d{1}|x\d+|001-|[\.\-\(\)])','',regex=True)
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())
print(us)


valid_phone
False    1205
Name: count, dtype: int64
      first_name last_name date_of_birth                        company  \
5          David    Torres    1949-08-12                   Yang-Stewart   
9          Emily     Jones    1992-10-09                Serrano-Leblanc   
16        Sierra   Spencer    1988-11-02       Watson, Davis and Cooper   
27         Bryan   Nielsen    1979-07-29                    Drake-Clark   
35       Melissa    Garcia    1946-06-30  Nelson, Williams and Martinez   
...          ...       ...           ...                            ...   
15275     George      Cook    1974-01-10                     Miller LLC   
15289    Stephen     Davis    1940-08-30      Graham, Fisher and Gibson   
15301    Stephen  Gonzales    2006-04-17                    Marquez Ltd   
15311    Stephen  Alvarado    2006-07-27                  Wallace-Smith   
15318    Stephen    Rivera    1952-06-04          Taylor, Fry and Jones   

                       email_address  \
5      

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3246566554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10


In [91]:
#Error check - ensure all numbers have exactly 10 digits
us.loc[:,'valid_phone'] = us['phone_number'].str.len() == 10
print(us['valid_phone'].value_counts())

valid_phone
False    1205
Name: count, dtype: int64


In [92]:
#Germany have 11 digits
germany = df[df['country'] == "Germany"]
print(germany.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4708 entries, 0 to 15317
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     4708 non-null   object        
 1   last_name      4708 non-null   object        
 2   date_of_birth  4708 non-null   datetime64[ns]
 3   company        4708 non-null   object        
 4   email_address  4708 non-null   object        
 5   address        4708 non-null   object        
 6   country        4708 non-null   object        
 7   country_code   4708 non-null   object        
 8   phone_number   2615 non-null   object        
 9   join_date      4708 non-null   datetime64[ns]
 10  user_uuid      4708 non-null   object        
dtypes: datetime64[ns](2), object(9)
memory usage: 441.4+ KB
None


In [93]:
# Some german numbers are 10 digits. Replace with NaN
germany.loc[:,'phone_number'] = germany['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)

In [94]:
#Error check - ensure all numbers have exactly 10 digits
germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11
germany.loc[~germany['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(germany['valid_phone'].value_counts())


valid_phone
True     2615
False    2093
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\787672041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  germany.loc[:,'valid_phone'] = germany['phone_number'].str.len() == 11


In [95]:
#Lastly the UK
uk = df[df['country'] == "United Kingdom"]
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

In [96]:
uk.loc[:,'phone_number'] = uk['phone_number'].str.replace(r'(\+\d{2})|\s+|[(|)]','', regex= True)
uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11
print(uk['valid_phone'].value_counts())

valid_phone
True     6474
False    2897
Name: count, dtype: int64


C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\1538497412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk.loc[:,'valid_phone'] = uk['phone_number'].str.len() == 11


In [97]:
uk.loc[~uk['phone_number'].apply(lambda x: len(str(x)) == 11), 'phone_number'] = np.nan
print(uk)

      first_name last_name date_of_birth                       company  \
1            Guy     Allen    1940-12-01                       Fox Ltd   
2          Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3         Darren   Hussain    1972-09-23                   Wheeler LLC   
4          Garry     Stone    1952-12-20                    Warner Inc   
6           Anne    Morris    1952-11-10                Hutchinson Inc   
...          ...       ...           ...                           ...   
15313    Stephen    Barker    1981-05-01             Marshall-Campbell   
15314    Stephen       Fox    1961-04-24             Williams-Phillips   
15315    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316    Stephen     Smith    1948-08-20               Robinson-Harris   
15319    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email_address  \
1      rhodesclifford@henderson.com   
2      glen98@bryant-marshall.co.

### Data cleaning of card details from the S3 bucket

In [4]:
import tabula

In [25]:
pdf = tabula.read_pdf('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf',stream = False, pages = 'all', guess = False, multiple_tables = True)

In [57]:
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

test = DataExtractor()
pdf = test.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

# tidy_card = DataCleaning(pdf)
# tidied_card = tidy_card.remove_question_mark()
# print(tidied_card)


In [34]:
print(type(pdf['card_number'][0]))

<class 'int'>


In [58]:
pdf['card_number'] = pdf['card_number'].astype(str).str.replace('?', '')
print(pdf)

            card_number expiry_date                card_provider  \
0        30060773296197       09/26  Diners Club / Carte Blanche   
1       349624180933183       10/23             American Express   
2      3529023891650490       06/23                 JCB 16 digit   
3       213142929492281       09/27                 JCB 15 digit   
4          502067329974       10/25                      Maestro   
...                 ...         ...                          ...   
15304   180036921556789       12/28                 JCB 15 digit   
15305   180018030448512       11/24                 JCB 15 digit   
15306  3569953313547220       04/24                 JCB 16 digit   
15307  4444521712606810       06/27                VISA 16 digit   
15308   372031786522735       02/30             American Express   

      date_payment_confirmed  
0                 2015-11-25  
1                 2001-06-18  
2                 2000-12-26  
3                 2011-02-12  
4                 1997-03-13

In [40]:
pdf[pdf['card_number'] == '4971858637664481']

,card_number,expiry_date,card_provider,date_payment_confirmed
149,4971858637664481,04/24,VISA 16 digit,2016-11-17


In [60]:
tidy_card = DataCleaning(pdf)
clean = tidy_card.clean_card_provider()

In [62]:
clean[clean.duplicated(subset=['card_number'], keep=False)]

,card_number,expiry_date,card_provider,date_payment_confirmed


In [63]:
clean.isnull().sum()

card_number               0
expiry_date               0
card_provider             0
date_payment_confirmed    0
dtype: int64

In [42]:
dup = pdf['card_number'].duplicated().any()
print(dup)

True


In [47]:
pdf.iloc[377]

card_number               NULL
expiry_date               NULL
card_provider             NULL
date_payment_confirmed    NULL
Name: 377, dtype: object

In [43]:
pdf[pdf.duplicated(subset=['card_number'], keep=False)]

,card_number,expiry_date,card_provider,date_payment_confirmed
377,NULL,NULL,NULL,NULL
847,NULL,NULL,NULL,NULL
884,NULL,NULL,NULL,NULL
2418,NULL,NULL,NULL,NULL
2489,NULL,NULL,NULL,NULL
2830,NULL,NULL,NULL,NULL
4196,NULL,NULL,NULL,NULL
5686,NULL,NULL,NULL,NULL
6024,NULL,NULL,NULL,NULL
12876,NULL,NULL,NULL,NULL


In [55]:
str_null = pdf.loc[pdf['card_number'] == 'NULL'].index.tolist()
pdf.drop(str_null, axis = 0, inplace = True)

In [56]:
pdf.loc[pdf['card_number'] == 'NULL']

,card_number,expiry_date,card_provider,date_payment_confirmed


In [22]:
print(len(pdf))
print(type(pdf[1]))

279
<class 'pandas.core.frame.DataFrame'>


In [12]:
pdf[1] 4971858637664481

,card_number,expiry_date,card_provider,date_payment_confirmed
0,372498568017103,07/28,American Express,2009-10-28
1,4108484730579010,03/28,VISA 16 digit,1994-08-21
2,676293500408,08/24,Maestro,1993-10-26
3,581681339767,06/29,Maestro,1998-08-03
4,180083117707156,06/30,JCB 15 digit,2012-01-11
5,213185617039336,04/23,JCB 15 digit,2005-03-28
6,180038452399167,09/27,JCB 15 digit,1999-11-04
7,213148437331713,06/28,JCB 15 digit,2004-02-11
8,6011852840796870,12/29,Discover,2020-10-14
9,6011662146447830,09/32,Discover,2013-04-22


In [10]:
import pandas as pd
pdf2 = pd.concat(pdf, ignore_index = True)
print(pdf2.head(5))
print(type(pdf2))

        card_number expiry_date                card_provider  \
0    30060773296197       09/26  Diners Club / Carte Blanche   
1   349624180933183       10/23             American Express   
2  3529023891650490       06/23                 JCB 16 digit   
3   213142929492281       09/27                 JCB 15 digit   
4      502067329974       10/25                      Maestro   

  date_payment_confirmed  
0             2015-11-25  
1             2001-06-18  
2             2000-12-26  
3             2011-02-12  
4             1997-03-13  
<class 'pandas.core.frame.DataFrame'>


In [17]:
'4971858637664481' in pdf2['card_number']

False

In [103]:
# Now in dataframe format
print(pdf2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB
None


In [104]:
card_names = pdf2['card_provider'].value_counts()
get_card_names = card_names[card_names > 1000].index.tolist()
print(get_card_names)

['VISA 16 digit', 'JCB 16 digit', 'VISA 13 digit', 'JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']


In [105]:
#All the dodgy cards have dodgy data, so remove
checking_dodgy_cards = pdf2[~pdf2['card_provider'].isin(get_card_names)]
print(checking_dodgy_cards)
print(checking_dodgy_cards.info())

      card_number expiry_date card_provider date_payment_confirmed
377          NULL        NULL          NULL                   NULL
827    VAB9DSB8ZM  NWS3P2W38H    NB71VBAHJE             GTC9KBWJO9
847          NULL        NULL          NULL                   NULL
884          NULL        NULL          NULL                   NULL
1443   MOZOT5Q95V  8YJ3TYH6Z5    WJVMUO4QX6             DJIXF1AFAZ
2418         NULL        NULL          NULL                   NULL
2489         NULL        NULL          NULL                   NULL
2830         NULL        NULL          NULL                   NULL
3694   K0084A9R99  ACT9K6ECRJ    JRPRLPIBZ2             H2PCQP4W50
4196         NULL        NULL          NULL                   NULL
4208   Y8ITI33X30  WDWMN9TU45    TS8A81WFXV             XTD27ANR5Q
4916   RNSCD8OCIM  VNLNMWPJII    JCQMU8FN85             7VGB4DA1WI
5686         NULL        NULL          NULL                   NULL
6024         NULL        NULL          NULL                   

In [106]:
# pdf3 symbolises the removal of nulls and dodgy data
pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]
print(pdf3['card_provider'].value_counts())

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [107]:
print(pdf3['expiry_date'])

0        09/26
1        10/23
2        06/23
3        09/27
4        10/25
         ...  
15304    12/28
15305    11/24
15306    04/24
15307    06/27
15308    02/30
Name: expiry_date, Length: 15284, dtype: object


In [108]:
#pdf3['expiry_date'] = pdf3['expiry_date'].apply(parse)
#pdf3['expiry_date'] = pd.to_datetime(pdf3['expiry_date'], errors='coerce', format = '%m/%Y')
#na_index = pdf3.index[pdf3['date_of_birth'].isna()].tolist() 
pdf3['expiry_date'].head(20)

0     09/26
1     10/23
2     06/23
3     09/27
4     10/25
5     11/23
6     07/27
7     02/29
8     02/24
9     07/23
10    10/28
11    11/27
12    11/31
13    10/29
14    01/29
15    01/26
16    02/32
17    10/26
18    08/28
19    09/32
Name: expiry_date, dtype: object

In [109]:
def custom_parse(date_string):
    month,year = date_string.split('/')
    return parse('01'+'/'+ date_string, ignoretz = True)

In [110]:
pdf3['expiry_date'][1]

'10/23'

In [111]:
pdf3.loc[:,'expiry_date'] = pdf3['expiry_date'].apply(custom_parse)
pdf3.loc[:,'expiry_date'] = pd.to_datetime(pdf3['expiry_date'])

In [112]:
print(pdf3['expiry_date'].head(20))
print(pdf3['expiry_date'].isnull().sum())


0     2026-01-09 00:00:00
1     2023-01-10 00:00:00
2     2023-01-06 00:00:00
3     2027-01-09 00:00:00
4     2025-01-10 00:00:00
5     2023-01-11 00:00:00
6     2027-01-07 00:00:00
7     2029-01-02 00:00:00
8     2024-01-02 00:00:00
9     2023-01-07 00:00:00
10    2028-01-10 00:00:00
11    2027-01-11 00:00:00
12    2031-01-11 00:00:00
13    2029-01-10 00:00:00
14    2029-01-01 00:00:00
15    2026-01-01 00:00:00
16    2032-01-02 00:00:00
17    2026-01-10 00:00:00
18    2028-01-08 00:00:00
19    2032-01-09 00:00:00
Name: expiry_date, dtype: object
0


In [113]:
#No expiry dates our of the ordinary
expired = pdf3[pdf3['expiry_date'] > pd.to_datetime('2050-01-01')]
print(expired)

Empty DataFrame
Columns: [card_number, expiry_date, card_provider, date_payment_confirmed]
Index: []


In [114]:
pdf3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15284 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15284 non-null  object
 3   date_payment_confirmed  15284 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [115]:
# date of payment looks good too
pdf3['date_payment_confirmed'].head(5)
pdf3.loc[:,'date_payment_confirmed'] = pdf3['date_payment_confirmed'].apply(parse)
pdf3['date_payment_confirmed'].isnull().sum()
pdf3[pdf3['date_payment_confirmed'] > pd.to_datetime("2022-12-01")]

,card_number,expiry_date,card_provider,date_payment_confirmed


In [116]:
# Card number
pdf3['card_number'].head(20)
print(type(pdf3['card_number'][1]))

<class 'int'>


In [117]:
def check_alpha(string):
    return str(string).isalpha()

In [118]:
#This didnt pick up some card numbers had '?' 
pdf3['card_number'].apply(check_alpha).value_counts()

card_number
False    15284
Name: count, dtype: int64

In [119]:
pdf3.loc[:,'card_number'] = str(pdf3['card_number'])
print(pdf3['card_provider'].value_counts())
# diners club = 14
# AMEX = 15
# Maestro = 12 to 19. 50, 56-69, 67
# Discover = 16
# Mastercard = 16

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64


In [120]:
def get_length(column):
    if 'Diners' in column:
        return 14
    elif 'American Express' in column:
        return 15
    elif 'Discover' in column:
        return 16
    elif 'Mastercard' in column:
        return 16
    elif 'JCB' in column or 'VISA' in column:
        return int(''.join(filter(str.isdigit, column)))
    elif 'Maestro' in column:
        return list(range(12,20))
    elif 'Diners Club / Carte Blanche' in column:
        return 14
    else:
        return None

In [121]:
pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3089059518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'expected_num_dig'] = pdf3['card_provider'].apply(get_length)


In [122]:
#Run this code if you encounter any errors 
#pdf3 = pdf2[pdf2['card_provider'].isin(get_card_names)]

In [123]:
pdf3[['card_number','card_provider','expected_num_dig']]

,card_number,card_provider,expected_num_dig
0,0 30060773296197\n1 349624180...,Diners Club / Carte Blanche,14
1,0 30060773296197\n1 349624180...,American Express,15
2,0 30060773296197\n1 349624180...,JCB 16 digit,16
3,0 30060773296197\n1 349624180...,JCB 15 digit,15
4,0 30060773296197\n1 349624180...,Maestro,"[12, 13, 14, 15, 16, 17, 18, 19]"
...,...,...,...
15304,0 30060773296197\n1 349624180...,JCB 15 digit,15
15305,0 30060773296197\n1 349624180...,JCB 15 digit,15
15306,0 30060773296197\n1 349624180...,JCB 16 digit,16
15307,0 30060773296197\n1 349624180...,VISA 16 digit,16


In [124]:
pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\284436329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3.loc[:,'comparison'] = pdf3.apply(lambda row: len(str(row['card_number'])) == row['expected_num_dig'] if isinstance(row['expected_num_dig'], int) else len(str(row['card_number'])) in row['expected_num_dig'], axis=1)


In [125]:
pdf3['comparison'].value_counts()

comparison
False    15284
Name: count, dtype: int64

In [126]:
import pandas as pd
review = pdf3[pdf3['comparison'] == False].head(100)
review.to_csv('review.csv')

In [127]:
review = pdf3[pdf3['card_provider'] == 'Maestro']
print(review)

                                             card_number          expiry_date  \
4      0          30060773296197\n1         349624180...  2025-01-10 00:00:00   
9      0          30060773296197\n1         349624180...  2023-01-07 00:00:00   
35     0          30060773296197\n1         349624180...  2027-01-08 00:00:00   
42     0          30060773296197\n1         349624180...  2027-01-07 00:00:00   
49     0          30060773296197\n1         349624180...  2025-01-02 00:00:00   
...                                                  ...                  ...   
15240  0          30060773296197\n1         349624180...  2032-01-04 00:00:00   
15285  0          30060773296197\n1         349624180...  2024-01-10 00:00:00   
15289  0          30060773296197\n1         349624180...  2026-01-11 00:00:00   
15290  0          30060773296197\n1         349624180...  2028-01-11 00:00:00   
15291  0          30060773296197\n1         349624180...  2028-01-05 00:00:00   

      card_provider date_pa

In [128]:
pdf3['card_number'] = pdf3['card_number'].astype(str)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_2852\3786287763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf3['card_number'] = pdf3['card_number'].astype(str)


In [129]:
pdf3[pdf3['card_number'].str.contains(r'\D')]


,card_number,expiry_date,card_provider,date_payment_confirmed,expected_num_dig,comparison
0,0 30060773296197\n1 349624180...,2026-01-09 00:00:00,Diners Club / Carte Blanche,2015-11-25 00:00:00,14,False
1,0 30060773296197\n1 349624180...,2023-01-10 00:00:00,American Express,2001-06-18 00:00:00,15,False
2,0 30060773296197\n1 349624180...,2023-01-06 00:00:00,JCB 16 digit,2000-12-26 00:00:00,16,False
3,0 30060773296197\n1 349624180...,2027-01-09 00:00:00,JCB 15 digit,2011-02-12 00:00:00,15,False
4,0 30060773296197\n1 349624180...,2025-01-10 00:00:00,Maestro,1997-03-13 00:00:00,"[12, 13, 14, 15, 16, 17, 18, 19]",False
...,...,...,...,...,...,...
15304,0 30060773296197\n1 349624180...,2028-01-12 00:00:00,JCB 15 digit,1997-06-06 00:00:00,15,False
15305,0 30060773296197\n1 349624180...,2024-01-11 00:00:00,JCB 15 digit,2004-06-16 00:00:00,15,False
15306,0 30060773296197\n1 349624180...,2024-01-04 00:00:00,JCB 16 digit,2020-02-05 00:00:00,16,False
15307,0 30060773296197\n1 349624180...,2027-01-06 00:00:00,VISA 16 digit,2008-06-16 00:00:00,16,False


In [130]:
pdf3.loc[pdf3['card_number'].str.contains(r'\D'), 'card_number'] = None

# Checking code to GET API data

In [51]:
#
import yaml
import requests

with open('db_creds.yaml','r') as file:
    credentials = yaml.safe_load(file)

response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=credentials['header'])

if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response Text: {response.text}")


In [52]:
print(data)

{'statusCode': 200, 'number_stores': 451}


In [53]:
import pandas as pd
import requests
import yaml

with open('db_creds.yaml', 'r') as file:
    credentials = yaml.safe_load(file)

data_list = []
for store_number in range(0, 451):
    response = requests.get(f"https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}", headers=credentials['header'])

    if response.status_code == 200:
        data = response.json()
        data_list.append(data)
        # Process the data as needed for each store
        print(f"Data for store {store_number}: {data}")
    else:
        print(f"Request for store {store_number} failed with status code: {response.status_code}")
        print(f"Response Text: {response.text}")

df_api = pd.DataFrame(data_list)

Data for store 0: {'index': 0, 'address': 'N/A', 'longitude': 'N/A', 'lat': 'N/A', 'locality': 'N/A', 'store_code': 'WEB-1388012W', 'staff_numbers': '325', 'opening_date': '2010-06-12', 'store_type': 'Web Portal', 'latitude': None, 'country_code': 'GB', 'continent': 'Europe'}
Data for store 1: {'index': 1, 'address': 'Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe', 'longitude': '51.62907', 'lat': None, 'locality': 'High Wycombe', 'store_code': 'HI-9B97EE4E', 'staff_numbers': '34', 'opening_date': '1996-10-25', 'store_type': 'Local', 'latitude': '-0.74934', 'country_code': 'GB', 'continent': 'Europe'}
Data for store 2: {'index': 2, 'address': 'Heckerstraße 4/5\n50491 Säckingen, Landshut', 'longitude': '48.52961', 'lat': None, 'locality': 'Landshut', 'store_code': 'LA-0772C7B9', 'staff_numbers': '92', 'opening_date': '2013-04-12', 'store_type': 'Super Store', 'latitude': '12.16179', 'country_code': 'DE', 'continent': 'Europe'}
Data for store 3: {'index': 3, 'address': '5 Har

# Reviewing and cleaning the API data

In [5]:
df_api.isnull().sum()

index              0
address            0
longitude          0
lat              440
locality           0
store_code         0
staff_numbers      0
opening_date       0
store_type         0
latitude           0
country_code       0
continent          0
dtype: int64

In [9]:
df_api['staff_numbers'] = pd.to_numeric(df_api['staff_numbers'], errors = 'coerce')
df_api[df_api['staff_numbers'] < 5]

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
84,85,"549 Watson highway\nMarcusport\nE2 3WZ, Exeter",50.7236,None,Exeter,EX-9ED6711E,4.0,1999-12-10,Mall Kiosk,-3.52751,GB,Europe
95,96,"Cichoriusplatz 61\n10993 Niesky, Verl",51.88333,None,Verl,VE-B81F9500,4.0,2000-07-10,Mall Kiosk,8.51667,DE,Europe
102,103,"64 Hayes crossing\nPort Ellieville\nG7S 1XQ, W...",51.84819,None,Walton-on-the-Naze,WA-FF3389AF,4.0,2001-02-16,Mall Kiosk,1.26738,GB,Europe
234,235,"23220 Christine Passage\nLake Karen, ME 66805,...",32.9156,None,Mira Mesa,MI-D57825AA,4.0,2000-02-12,Mall Kiosk,-117.14392,US,America
252,253,"5 Gill square\nBlakeborough\nPR9W 6XB, Brierle...",52.48173,None,Brierley Hill,BR-FB62A5BA,4.0,2004-12-30,Mall Kiosk,-2.12139,GB,Europe
284,285,"Reinhard-Weller-Allee 7\n09791 Uelzen, Reutlingen",48.49144,None,Reutlingen,RE-FAF57EF7,4.0,2013-07-13,Mall Kiosk,9.20427,DE,eeEurope
298,299,"05 Stone trail\nLake Alexandratown\nW3 1YU, Bl...",53.81667,None,Blackpool,BL-D6CF153F,4.0,2004-03-15,Mall Kiosk,-3.05,GB,Europe
299,300,"Flat 12\nHarriet circle\nSandraland\nUB0V 3QF,...",56.56317,None,Arbroath,AR-2284FD32,4.0,2018-11-29,Mall Kiosk,-2.58736,GB,Europe
315,316,"118 Logan Port Suite 682\nNew Jacobton, SC 818...",28.15112,None,Lutz,LU-439557A9,4.0,2012-08-18,Mall Kiosk,-82.46148,US,America
330,331,"Flat 9\nMarc islands\nJoanfort\nLN39 1RY, Bushey",51.64316,None,Bushey,BU-79F72A64,4.0,2011-09-24,Mall Kiosk,-0.36053,GB,Europe


In [10]:
df_api['store_code'].value_counts()

store_code
NULL           3
HI-9B97EE4E    1
BL-9A86B74D    1
AB-30E0C17F    1
HO-16A376B0    1
              ..
LY-89B9F675    1
SU-5F78B30A    1
CL-DA5365DF    1
ME-9940FF73    1
BA-B4AED588    1
Name: count, Length: 448, dtype: int64

In [15]:
df_api['store_code'].head(50)

0     HI-9B97EE4E
1     LA-0772C7B9
2     WE-1DE82CEE
3     BE-18074576
4     GA-CAD01AC2
5     RU-C603E990
6     ST-229D997E
7     KA-FA7ED3B8
8     HA-974352FE
9     RU-9F1136B4
10    SI-ECD52CD9
11    DE-585399CF
12    CR-792AA8BB
13    HA-39A446E2
14    LA-9B0D9277
15    NE-1D8B1D0C
16    LY-4C3D5D6C
17    CH-6A561423
18    BA-898BDED3
19    EA-77ECA680
20    PO-38790FAE
21    BU-251A0E5A
22    EH-91356030
23    AB-917B715E
24    SU-0B4C9A5F
25    IN-157E1191
26    GA-DA8EEA4A
27    LA-2B59A825
28    WE-31C8B335
29    LA-F1042C48
30    EA-24B31935
31    EA-7965E06D
32    PE-040B15C3
33    EH-DB8676C1
34    AL-91D3A03C
35    HE-E39F4BC6
36    PO-47A01287
37    LA-D78C5F3F
38    ME-31958763
39    AR-5E72668B
40    CO-CB3D8C89
41    CL-5C7C3198
42    BO-17E7B6CE
43    CH-619E036C
44    BR-BC499EDD
45    NE-E50207AD
46    BE-8C0CF738
47    NE-374D3983
48    SU-95D20AE9
49    ME-FB62E459
Name: store_code, dtype: object

In [44]:
import re

# Assuming df_api is your DataFrame
pattern = r'^(?!\w{2}-\w{8}$).*$'
filtered_df = df_api[df_api['store_code'].astype(str).str.match(pattern, na=False)]
filtered_df.head(50)

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,first_two_letters
62,63,6FWDZHD7PW,1ZVU03X2P6,13KJZ890JH,9IBH8Y4Z0S,NRQKZWJ9OZ,-1.0,ZCXWWKF45G,QP74AHEQT0,1CJ5OAU4BR,YELVM536YT,QMAVR5H3LD,NR
171,172,UBCIFQLSNY,Q1TJY8H1ZH,2XE1OWOC23,1T6B406CI8,QIUU9SVP51,-1.0,7AHXLXIUEF,O0QJIRC943,3ZR3F89D97,FP8DLXQVGH,LU3E036ZD9,QI
216,217,NULL,NULL,NULL,NULL,NULL,-1.0,NULL,NULL,NULL,NULL,NULL,NU
230,231,RC99UKMZB2,2YBZ1440V6,OXVE5QR07O,6LVWPU1G64,Y8J0Z2W8O9,-1.0,0OLAK2I6NS,50IB01SFAZ,L13EQEQODP,HMHIFNLOBN,5586JCLARW,Y8
332,333,X349GIDWKU,O7NF1FZ74Y,VKA5I8H32X,RX9TCP2RGB,ISEE8A57FE,-1.0,A3PMVM800J,0RSNUU3DF5,J3BPB68Z1J,F3AO8V2LHU,GFJQ2AAEQ8,IS
380,381,ZBGB54ID4H,SKO4NMRNNF,LACCWDI0SB,CQMHKI78BX,T0R2CQBDUS,-1.0,GMMB02LA9V,B4KVQB3P5Y,AJHOMDOHZ4,OH20I92LX3,SLQBD982C0,T0
404,405,NULL,NULL,NULL,NULL,NULL,-1.0,NULL,NULL,NULL,NULL,NULL,NU
413,414,XTUAV57DP4,ID819KG3X5,A3O5CBWAMD,RY6K0AUE7F,TUOKF5HAAQ,-1.0,13PIY8GD1H,X0FE7E2EOG,AE7EEW4HSS,OYVW925ZL8,XQ953VS0FG,TU
436,437,NULL,NULL,NULL,NULL,NULL,-1.0,NULL,NULL,NULL,NULL,NULL,NU
446,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,-1.0,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL,9D


In [16]:
df_api['first_two_letters'] = df_api['store_code'].apply(lambda x: x[:2] if isinstance(x, str) else None)

In [24]:
value_counts = df_api['first_two_letters'].value_counts()
print(value_counts)

first_two_letters
CH    22
LA    18
WE    15
BE    15
BU    15
      ..
BO     1
Y8     1
NR     1
MO     1
9D     1
Name: count, Length: 80, dtype: int64


In [39]:
# Filter the indices where the value counts are less than 10
indices_less_than_10 = value_counts.index[value_counts < 1].tolist()

# Display the indices
print(indices_less_than_10)

[]


In [35]:
df_api[df_api['first_two_letters'].isin(indices_less_than_10)].head(50)

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,first_two_letters
7,8,"510 Jill Mill\nSouth Laura, FL 38723, Kaukauna",44.27804,None,Kaukauna,KA-FA7ED3B8,31.0,2022-09-05,Local,-88.27205,US,America,KA
10,11,"Bergerstr. 1/3\n59183 Seelow, Siegburg",50.80019,None,Siegburg,SI-ECD52CD9,25.0,July 2015 14,Local,7.20769,DE,Europe,SI
11,12,Flat 37\nBennett expressway\nNew Charlotte\nSY...,51.35084,None,Devizes,DE-585399CF,36.0,2014-10-11,Local,-1.99421,GB,Europe,DE
12,13,"520 Fisher Inlet Suite 594\nPort Eric, IL 7175...",42.24113,None,Crystal Lake,CR-792AA8BB,138.0,2018-04-05,Super Store,-88.3162,US,America,CR
42,43,"92863 Kathryn Ford Suite 747\nEast Craig, DC 2...",42.35843,None,Boston,BO-17E7B6CE,75.0,2019-07-28,Super Store,-71.05977,US,America,BO
50,51,"Mosemannstr. 81\n30877 Schrobenhausen, Karben",50.23019,None,Karben,KA-653E783F,36.0,1998-07-07,Local,8.77155,DE,Europe,KA
51,52,"220 Holt Unions Suite 688\nRamirezstad, IL 749...",39.45621,None,Martinsburg,MA-F0E23355,8.0,2011-07-07,Mall Kiosk,-77.96389,US,America,MA
52,53,"Trommlerstr. 7/5\n75372 Wolfach, Oschatz",51.30001,None,Oschatz,OS-70B2CD28,36.0,2007-02-04,Local,13.10984,DE,Europe,OS
60,61,"Martha-Dörr-Platz 1/3\n53035 Bad Mergentheim, ...",49.13645,None,Eppingen,EP-105C3FB5,27.0,2014-07-16,Local,8.91229,DE,Europe,EP
62,63,6FWDZHD7PW,1ZVU03X2P6,13KJZ890JH,9IBH8Y4Z0S,NRQKZWJ9OZ,NaN,ZCXWWKF45G,QP74AHEQT0,1CJ5OAU4BR,YELVM536YT,QMAVR5H3LD,NR


In [40]:
# Assuming df_api is your DataFrame
df_api['staff_numbers'] = df_api['staff_numbers'].fillna(-1)  # Filling null values with a specific value, -1 in this case
df_api[df_api['staff_numbers'] < 1]

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,first_two_letters
30,31,"Flat 69\nSuzanne walk\nEast Michelle\nE80 8HS,...",52.68333,None,East Dereham,EA-24B31935,-1.0,2012-11-09,Outlet,0.93333,GB,Europe,EA
62,63,6FWDZHD7PW,1ZVU03X2P6,13KJZ890JH,9IBH8Y4Z0S,NRQKZWJ9OZ,-1.0,ZCXWWKF45G,QP74AHEQT0,1CJ5OAU4BR,YELVM536YT,QMAVR5H3LD,NR
171,172,UBCIFQLSNY,Q1TJY8H1ZH,2XE1OWOC23,1T6B406CI8,QIUU9SVP51,-1.0,7AHXLXIUEF,O0QJIRC943,3ZR3F89D97,FP8DLXQVGH,LU3E036ZD9,QI
178,179,"Girschnerweg 163\n93597 Angermünde, Dahlem",52.4581,None,Dahlem,DA-ACC520AE,-1.0,1994-03-07,Local,13.28702,DE,Europe,DA
216,217,NULL,NULL,NULL,NULL,NULL,-1.0,NULL,NULL,NULL,NULL,NULL,NU
230,231,RC99UKMZB2,2YBZ1440V6,OXVE5QR07O,6LVWPU1G64,Y8J0Z2W8O9,-1.0,0OLAK2I6NS,50IB01SFAZ,L13EQEQODP,HMHIFNLOBN,5586JCLARW,Y8
247,248,"7430 Howe Extensions Suite 299\nKellyside, WA ...",39.71734,None,Sicklerville,SI-C489938D,-1.0,1994-02-28,Outlet,-74.96933,US,America,SI
332,333,X349GIDWKU,O7NF1FZ74Y,VKA5I8H32X,RX9TCP2RGB,ISEE8A57FE,-1.0,A3PMVM800J,0RSNUU3DF5,J3BPB68Z1J,F3AO8V2LHU,GFJQ2AAEQ8,IS
340,341,Studio 8\nLydia groves\nNorth Hilarymouth\nIV4...,50.79205,None,Southsea,SO-B5B9CB3B,-1.0,2018-05-08,Super Store,-1.08593,GB,Europe,SO
374,375,"Salzstraße 1/9\n74209 Bad Freienwalde, Charlot...",52.53048,None,Charlottenburg-Nord,CH-99475026,-1.0,1995-03-05,Local,13.29371,DE,Europe,CH


In [49]:
contains_null = df_api[df_api['address'].isnull()]
print(contains_null)

Empty DataFrame
Columns: [index, address, longitude, lat, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent, first_two_letters]
Index: []


In [50]:
df_api.to_csv("api_data.csv")

In [346]:
api_data = pd.read_csv("api_data.csv")

In [347]:
api_data.head(2)

,Unnamed: 0,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,NaN,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
1,1,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,NaN,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe


In [348]:
api_data =api_data.drop(api_data.columns[0],axis=1)

In [349]:
# Is lat an irrelevant empty column? yes
api_data['lat'].value_counts()

lat
13KJZ890JH    1
2XE1OWOC23    1
OXVE5QR07O    1
VKA5I8H32X    1
LACCWDI0SB    1
A3O5CBWAMD    1
UXMWDMX1LC    1
Name: count, dtype: int64

In [350]:
api_data =api_data.drop('lat',axis = 1)

In [351]:
#Randomly generated rows exist, use a numbers column to dig these out
def check_alpha(string):
    return str(string).isalpha()

random = api_data[api_data['country_code'].str.len() > 2]

In [146]:
print(random)

     Unnamed: 0  index     address   longitude         lat    locality  \
62           62     63  6FWDZHD7PW  1ZVU03X2P6  13KJZ890JH  9IBH8Y4Z0S   
171         171    172  UBCIFQLSNY  Q1TJY8H1ZH  2XE1OWOC23  1T6B406CI8   
230         230    231  RC99UKMZB2  2YBZ1440V6  OXVE5QR07O  6LVWPU1G64   
332         332    333  X349GIDWKU  O7NF1FZ74Y  VKA5I8H32X  RX9TCP2RGB   
380         380    381  ZBGB54ID4H  SKO4NMRNNF  LACCWDI0SB  CQMHKI78BX   
413         413    414  XTUAV57DP4  ID819KG3X5  A3O5CBWAMD  RY6K0AUE7F   
446         446    447  K0ODETRLS3  K8CXLZDP07  UXMWDMX1LC  3VHFDNP8ET   

     store_code staff_numbers opening_date  store_type    latitude  \
62   NRQKZWJ9OZ    BIP8K8JJW2   ZCXWWKF45G  QP74AHEQT0  1CJ5OAU4BR   
171  QIUU9SVP51    SKBXAXF5G5   7AHXLXIUEF  O0QJIRC943  3ZR3F89D97   
230  Y8J0Z2W8O9    2429OB3LMM   0OLAK2I6NS  50IB01SFAZ  L13EQEQODP   
332  ISEE8A57FE    74BY7HSB6P   A3PMVM800J  0RSNUU3DF5  J3BPB68Z1J   
380  T0R2CQBDUS    GT1FO6YGD4   GMMB02LA9V  B4KVQB3P5Y  A

In [352]:
# These rows have been found and will be dropped
api_data = api_data[api_data['country_code'].str.len() <= 2]

In [164]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          440 non-null    int64 
 1   address        440 non-null    object
 2   longitude      440 non-null    object
 3   locality       440 non-null    object
 4   store_code     440 non-null    object
 5   staff_numbers  440 non-null    object
 6   opening_date   440 non-null    object
 7   store_type     440 non-null    object
 8   latitude       440 non-null    object
 9   country_code   440 non-null    object
 10  continent      440 non-null    object
dtypes: int64(1), object(10)
memory usage: 41.2+ KB


In [233]:
#no nulls
api_data.isnull().sum()

index            0
address          0
longitude        0
locality         0
store_code       0
staff_numbers    0
opening_date     0
store_type       0
latitude         0
country_code     0
continent        0
dtype: int64

In [291]:
#Remove new line from address and replace with commas
api_data['address'] = api_data['address'].str.replace('\n', ',')

In [292]:
api_data['locality'].value_counts()

locality
Chapletown       14
Belper           13
Bushey           12
Exeter           11
Arbroath         10
                 ..
Sicklerville      1
Brierley Hill     1
Searcy            1
Port Richmond     1
Westchester       1
Name: count, Length: 116, dtype: int64

In [293]:
#Store code looks good
print(api_data['store_code'])
pattern = r'^[A-Z]{2}-[0-9A-F]{8}$'
unmatched_strings = api_data[~api_data['store_code'].str.contains(pattern)]


0      HI-9B97EE4E
1      LA-0772C7B9
2      WE-1DE82CEE
3      BE-18074576
4      GA-CAD01AC2
          ...     
444    CO-473A9FBB
445    KI-78096E8C
447    HI-EEA7AE62
448    BR-662EC74C
449    BA-B4AED588
Name: store_code, Length: 440, dtype: object


In [294]:
print(unmatched_strings)

Empty DataFrame
Columns: [index, address, longitude, locality, store_code, staff_numbers, opening_date, store_type, latitude, country_code, continent]
Index: []


In [295]:
# Opening date
date_check = api_data.copy()

In [296]:
date_check['opening_date']

0      1996-10-25
1      2013-04-12
2      2014-01-02
3      2019-09-09
4      1995-05-15
          ...    
444    2008-06-08
445    2005-05-12
447    1998-05-14
448    2020-10-17
449    2001-05-12
Name: opening_date, Length: 440, dtype: object

In [306]:
date_check['opening_date'] = pd.to_datetime(date_check['opening_date'], format = 'mixed')

In [308]:
null_index = date_check.loc[date_check['opening_date'].isnull()].index

In [309]:
api_data.loc[null_index]['opening_date']

Series([], Name: opening_date, dtype: object)

In [307]:
date_check.loc[null_index]['opening_date']

9     2012-10-08
10    2015-07-14
93    2022-01-20
121   2020-02-01
142   2003-05-27
189   2016-11-25
205   2008-12-07
241   2006-10-04
291   2001-05-04
339   1994-11-24
368   2009-02-28
393   2015-03-02
Name: opening_date, dtype: datetime64[ns]

In [310]:
api_data['opening_date'] = pd.to_datetime(api_data['opening_date'], format = 'mixed')

In [311]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          440 non-null    int64         
 1   address        440 non-null    object        
 2   longitude      440 non-null    object        
 3   locality       440 non-null    object        
 4   store_code     440 non-null    object        
 5   staff_numbers  440 non-null    object        
 6   opening_date   440 non-null    datetime64[ns]
 7   store_type     440 non-null    object        
 8   latitude       440 non-null    object        
 9   country_code   440 non-null    object        
 10  continent      440 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 57.4+ KB


In [312]:
staff_number = api_data.copy()

In [325]:
#Throws error letters present
staff_number['staff_numbers'].astype(int)

0      34
1      92
2      69
3      35
4      36
       ..
444    94
445    61
447    33
448    35
449    36
Name: staff_numbers, Length: 440, dtype: int32

In [322]:
non_digit_records = staff_number[staff_number['staff_numbers'].str.contains(r'\D', regex=True)]
print(non_digit_records)

     index                                            address longitude  \
30      31  Flat 69,Suzanne walk,East Michelle,E80 8HS, Ea...  52.68333   
178    179          Girschnerweg 163,93597 Angermünde, Dahlem   52.4581   
247    248  7430 Howe Extensions Suite 299,Kellyside, WA 9...  39.71734   
340    341  Studio 8,Lydia groves,North Hilarymouth,IV4 6E...  50.79205   
374    375  Salzstraße 1/9,74209 Bad Freienwalde, Charlott...  52.53048   

                locality   store_code staff_numbers opening_date   store_type  \
30          East Dereham  EA-24B31935           J78   2012-11-09       Outlet   
178               Dahlem  DA-ACC520AE           30e   1994-03-07        Local   
247         Sicklerville  SI-C489938D           80R   1994-02-28       Outlet   
340             Southsea  SO-B5B9CB3B           A97   2018-05-08  Super Store   
374  Charlottenburg-Nord  CH-99475026           3n9   1995-03-05        Local   

      latitude country_code continent  
30     0.93333        

In [319]:
# didnt pick up that staff numbers had lettters
staff_number[staff_number['staff_numbers'].astype(str).str.len() > 3]

,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


In [324]:
# Remove letters but keep the numbers in 
api_data['staff_numbers'] = api_data['staff_numbers'].replace(r'\D', '', regex=True)

In [326]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          440 non-null    int64         
 1   address        440 non-null    object        
 2   longitude      440 non-null    object        
 3   locality       440 non-null    object        
 4   store_code     440 non-null    object        
 5   staff_numbers  440 non-null    object        
 6   opening_date   440 non-null    datetime64[ns]
 7   store_type     440 non-null    object        
 8   latitude       440 non-null    object        
 9   country_code   440 non-null    object        
 10  continent      440 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 57.4+ KB


In [327]:
#Store type looks good
api_data['store_type'].value_counts()

store_type
Local          255
Super Store     89
Mall Kiosk      51
Outlet          45
Name: count, dtype: int64

In [328]:
255 + 89 +51 +45

440

In [334]:
api_data['latitude'].astype(float)


count          440
unique         116
top       -1.47217
freq            14
Name: latitude, dtype: object

In [353]:
api_data['longitude'].astype(float)

0      51.62907
1      48.52961
2      51.26000
3      53.02330
4      53.38333
         ...   
444    50.76306
445    52.20000
447    51.62907
448    49.03685
449    50.64336
Name: longitude, Length: 440, dtype: float64

In [335]:
api_data['country_code'].value_counts()

country_code
GB    265
DE    141
US     34
Name: count, dtype: int64

In [336]:
api_data['country_code'].value_counts().sum()

440

In [340]:
api_data['continent'].value_counts()

continent
Europe     406
America     34
Name: count, dtype: int64

In [339]:
api_data['continent'] = api_data['continent'].str.replace('ee', '')

### Extracting S3 object using boto3

In [2]:
# Downloading S3 object using boto3
# import boto3
# s3 = boto3.client('s3')
# s3.download_file('data-handling-public','products.csv','D:/vs_ipynb/learning-material/multinational-retail/products.csv')

In [3]:
# import pandas as pd
# df_csv = pd.read_csv('products.csv',index_col = 0)

In [11]:
print(df_csv)

                                     product_name product_price  weight  \
0     FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   1.6kg   
1             Tiffany's World Day Out At The Park        £12.99  0.48kg   
2             Tiffany's World Pups Picnic Playset         £7.00    590g   
3        Tiffany's World Wildlife Park Adventures        £12.99    540g   
4                         Cosatto Cosy Dolls Pram        £30.00  1.91kg   
...                                           ...           ...     ...   
1848    Goodmans Qi Autosense In Car Phone Holder        £15.00    125g   
1849    Goodmans Qi Wireless Charging Phone Stand        £15.00    134g   
1850                     RAC 12V Wet & Dry Vacuum        £18.00    820g   
1851    Goodmans Qi Autosense In Car Phone Holder        £15.00    125g   
1852               Beldray Telescopic Ladder 2.9m        £69.00   9.4kg   

            category            EAN  date_added  \
0     toys-and-games  7425710935115  2005-12-02 

In [2]:
import boto3
from io import BytesIO
import pandas as pd

# Initialize Boto 3 S3 client
s3 = boto3.client('s3')

# Define the bucket and object key
bucket_name = 'data-handling-public'
object_key = 'products.csv'

# Read the file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    # Access and read the content of the file. Data type = Bytes due to get request
    data = response['Body'].read()

    # Process the data as needed
    df = pd.read_csv(BytesIO(data), index_col = 0)  # Replace this with your processing logic

except Exception as e:
    print(f"Error: {e}")

In [1]:
# Example S3 path
s3_path = "s3://data-handling-public/products.csv"

# Separating bucket and object
if s3_path.startswith("s3://"):
    path_without_protocol = s3_path[len("s3://"):]
    bucket, obj = path_without_protocol.rsplit('/', 1)

    print("Bucket:", bucket)
    print("Object:", obj)
else:
    print("Invalid S3 path.")

Bucket: data-handling-public
Object: products.csv


In [4]:
print(df.head(15))

                                   product_name product_price   weight  \
0   FurReal Dazzlin' Dimples My Playful Dolphin        £39.99    1.6kg   
1           Tiffany's World Day Out At The Park        £12.99   0.48kg   
2           Tiffany's World Pups Picnic Playset         £7.00     590g   
3      Tiffany's World Wildlife Park Adventures        £12.99     540g   
4                       Cosatto Cosy Dolls Pram        £30.00   1.91kg   
5                  Cocomelon Magnetic Scribbler        £12.99   0.91kg   
6                       Peppa Pig House Felties         £7.00   0.46kg   
7                     Paw Patrol Walkie Talkies        £12.99   0.38kg   
8                   Peppa Pig 3D Walkie Talkies        £12.99   0.38kg   
9          Disney Encanto Magical Casa Madrigal        £89.99  8.981kg   
10                Disney Encanto Mirabel's Room        £24.99  1.478kg   
11         Disney Encanto Isabela's Garden Room        £24.99     1.2g   
12          Disney Encanto Antonio's T

In [30]:
# Check for null
df.info() # 4

<class 'pandas.core.frame.DataFrame'>
Index: 1853 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1849 non-null   object
 1   product_price  1849 non-null   object
 2   weight         1849 non-null   object
 3   category       1849 non-null   object
 4   EAN            1849 non-null   object
 5   date_added     1849 non-null   object
 6   uuid           1849 non-null   object
 7   removed        1849 non-null   object
 8   product_code   1849 non-null   object
dtypes: object(9)
memory usage: 144.8+ KB


In [54]:
# null_values = df[df.isnull().any(axis=1)].index
# df.drop(null_values)
df.dropna(inplace=True)

In [53]:
# Likely to be random generated data
df['category'].value_counts()

category
homeware              1138
toys-and-games         257
food-and-drink         250
pets                    93
sports-and-leisure      40
health-and-beauty       40
diy                     28
Name: count, dtype: int64

In [49]:
cat = df['category'].value_counts()
get_cat = cat[cat < 2].index.tolist()
df[df['category'].isin(get_cat)]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code


In [48]:
df = df[~df['category'].isin(get_cat)]
print(df['category'].value_counts())

category
homeware              1138
toys-and-games         257
food-and-drink         250
pets                    93
sports-and-leisure      40
health-and-beauty       40
diy                     28
Name: count, dtype: int64


In [36]:
# Find rows and columns containing '?'. Doesnt exist for once
question_mark_indices = df.map(lambda x: '?' in str(x)).any()
question_mark_rows = df[df.map(lambda x: '?' in str(x)).any(axis=1)].index

print("Columns with '?':")
print(question_mark_indices[question_mark_indices].index.tolist())
print("Rows with '?':")
print(question_mark_rows.tolist())

Columns with '?':
[]
Rows with '?':
[]


In [44]:
# null_values = df[df.isnull().any(axis=1)].index
# print(null_values)

Index([], dtype='int64')


In [55]:
df[['weight','units']] = df['weight'].str.extract('(\d*\.?\d+)(\D+)', expand=True)

In [51]:
df['units'].value_counts()

units
kg     954
g      853
 x      28
ml       9
g .      1
oz       1
Name: count, dtype: int64

In [70]:
df.loc[df.index[df['units'].str.contains('x', case=True, na=False)].tolist()]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,units
298,Pedigree Vital Protection Dog Food in Jelly 12...,£4.49,12,pets,2439834307647,1995-06-25,5ec5a431-7330-4d9e-bf3c-702fc85f6efe,Still_avaliable,d4-9698287C,x
299,Cesar Country Kitchen Dog Food 8 x 150g,£5.49,8,pets,5158002633117,1994-03-22,c379d810-61bf-4ed2-994c-0d3308d5be7b,Still_avaliable,M4-1688574Q,x
300,Pedigree Puppy Dog Food in Jelly 12pk,£4.49,12,pets,5242405808651,2014-05-20,40963728-bdae-48f1-beec-7c6dd0225921,Still_avaliable,J4-5517838n,x
301,Pedigree Dog Food in Gravy 12pk,£4.49,12,pets,6160542808008,2020-12-20,3109d9e8-56b5-4233-a682-0bcdb38182b0,Still_avaliable,v6-1209149g,x
302,Pedigree Dog Food in Jelly 12pk,£4.49,12,pets,3928398867180,2006-03-27,28625483-3723-4eaa-9f14-a1a16affbed8,Still_avaliable,C5-0179057G,x
303,Chappie Dog Food 6 x 412g,£5.99,6,pets,8155594260063,1993-06-30,6b86f83c-fccf-42eb-b1d5-3971915c18a5,Still_avaliable,Z4-535642u,x
304,Butcher's Puppy Meaty Chunks in Jelly Tins 6 x...,£4.35,6,pets,9386802556862,2019-06-16,a21e8202-de56-40bf-aee9-1b65989b1030,Still_avaliable,x0-3876046p,x
305,Pedigree Vital Protection Dog Food in Jelly 12...,£4.49,12,pets,6045981116269,1996-10-12,39ce65ab-d873-4953-a363-af9a705ca1ef,Still_avaliable,O3-6314460b,x
381,Sheba Classics Ocean Selection in Terrine 8 x 85g,£3.49,8,pets,5864355297349,2014-01-01,96f4d10f-1cb0-4898-b170-8dcd54c78f05,Still_avaliable,I3-7208040s,x
382,Classic Jumbo Pack Cat Food in Jelly 40 x 100g,£10.99,40,pets,3610175997493,2003-09-03,003e2c32-2d88-4ace-9d3c-97863fb62959,Still_avaliable,Y9-9993954b,x


In [39]:
def transform_weight(row):
    weight, unit = row['weight'],row['units']
    if unit == 'kg':
        return weight
    elif unit == 'g'or unit == 'g .':
        return weight / 1000 
    elif unit == 'ml':
        return weight / 1000
    elif unit == 'oz':
        return weight * 0.0283495
    else:
        return 0

In [56]:
df['weight'] = df.apply(transform_weight,axis=1)

In [43]:
df['weight'] = (df['weight']).astype(float)

C:\Users\Janoh\AppData\Local\Temp\ipykernel_18948\3355173983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = (df['weight']).astype(float)


In [44]:
print(df)

                                     product_name product_price  weight  \
0     FurReal Dazzlin' Dimples My Playful Dolphin        £39.99     0.0   
1             Tiffany's World Day Out At The Park        £12.99     0.0   
2             Tiffany's World Pups Picnic Playset         £7.00     0.0   
3        Tiffany's World Wildlife Park Adventures        £12.99     0.0   
4                         Cosatto Cosy Dolls Pram        £30.00     0.0   
...                                           ...           ...     ...   
1848    Goodmans Qi Autosense In Car Phone Holder        £15.00     0.0   
1849    Goodmans Qi Wireless Charging Phone Stand        £15.00     0.0   
1850                     RAC 12V Wet & Dry Vacuum        £18.00     0.0   
1851    Goodmans Qi Autosense In Car Phone Holder        £15.00     0.0   
1852               Beldray Telescopic Ladder 2.9m        £69.00     0.0   

            category            EAN  date_added  \
0     toys-and-games  7425710935115  2005-12-02 

In [86]:
df.drop('units', axis=1, inplace = True)

In [96]:
# More NAs
null_rows = df[df.isnull().any(axis=1)].index
print(null_rows)

Index([751, 1133, 1400], dtype='int64')


In [100]:
df.iloc[[750, 1130, 1397]]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
751,VLPCU81M30,XCD69KUI0K,0,NaN,OO7KH8P79I,CCAVRB79VV,7QB0Z9EW1G,T3QRRH7SRP,SDAV678FVD
1133,9SX4G65YUX,N9D2BZQX63,0,NaN,E8EOGWOY8S,09KREHTMWL,CP8XYQVGGU,BPSADIOQOK,BSDTR67VD90
1400,LB3D71C025,ODPMASE7V7,0,NaN,BHPF2JTNKQ,PEPWA0NCVH,VIBLHHVPMN,H5N71TV8AY,OPSD21HN67


In [101]:
df.drop(null_rows, axis = 0, inplace = True)

In [102]:
df['date_added'] = pd.to_datetime(df['date_added'], format = 'mixed')

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1846 non-null   object        
 1   product_price  1846 non-null   object        
 2   weight         1846 non-null   float64       
 3   category       1846 non-null   object        
 4   EAN            1846 non-null   object        
 5   date_added     1846 non-null   datetime64[ns]
 6   uuid           1846 non-null   object        
 7   removed        1846 non-null   object        
 8   product_code   1846 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 144.2+ KB


In [106]:
df['removed'].value_counts()

removed
Still_avaliable    1752
Removed              94
Name: count, dtype: int64

In [109]:
df[~df['product_price'].str.contains('£', na=False)]

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code


### Tidy order table

In [5]:
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

connect = DatabaseConnector()
connect.read_db_creds()
connect.init_db_engine()
connect.list_db_tables()

extract = DataExtractor()
table = extract.read_rds_table(connect, 'orders_table')
print(table)

legacy_store_details
legacy_users
orders_table
        level_0   index                             date_uuid first_name  \
0             0       0  9476f17e-5d6a-4117-874d-9cdb38ca1fa6       None   
1             1       1  0423a395-a04d-4e4a-bd0f-d237cbd5a295       None   
2             2       2  65187294-bb16-4519-adc0-787bbe423970       None   
3             3       3  579e21f7-13cb-436b-83ad-33687a4eb337       None   
4             4       4  00ab86c3-2039-4674-b9c1-adbcbbf525bd       None   
...         ...     ...                                   ...        ...   
120118   110549  110548  f0e8fff6-9998-4661-954b-0e258e09d33c       None   
120119    82164   82164  1c80940a-d186-4ba9-9daa-8abd1aceae32       None   
120120    97599   97599  58598aca-049c-418e-8e39-46327634a7f1     Sharon   
120121   106591  106591  3a76f661-0707-4fbc-9862-f21d3249f581       None   
120122   118806  118804  98fdc8b3-f619-4471-86b4-f44c26880166       None   

       last_name                        

In [6]:
table.loc[table['card_number'] == 4971858637664481]

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
182,149,149,7559e9a1-a844-47b5-91af-d1339d068fd0,None,None,e74907ca-1a4a-476c-a3ca-6b898b0964c2,4971858637664481,NI-AB0FECD7,L7-4325310J,NaN,4
120062,31765,31765,a95d389d-c869-4f45-8177-8e6b22693daf,None,None,ab1091f6-e28a-4750-931b-b5707c5ab701,4971858637664481,VE-EE7A1F48,P0-4021587w,NaN,4
120063,37717,37717,0f8ff48c-0a74-4982-b002-45bdac0fd00f,None,None,4213c560-e4c2-4b80-8b7c-f64a447fb242,4971858637664481,KI-78096E8C,H7-3096678F,NaN,5
120077,43182,43181,0fe03a4d-d602-483a-a2b0-89e0e9ac186b,None,None,2e038121-c3f1-470a-aa31-710f4cedd639,4971858637664481,MI-20EE1BFA,A0-199587D,NaN,2
120078,44555,44555,47c0919b-a078-4dd7-b194-ac6923c00b0b,None,None,528f01e6-c0bb-41e1-a533-a87dd264708e,4971858637664481,GA-5957F561,K2-8569126h,NaN,6
120106,53055,53054,826fcd34-2886-493b-9f05-071c0c73bb8c,Lee,Thomas,a2ddc7d8-3434-4a3d-920f-17befdee38a3,4971858637664481,WEB-1388012W,V2-1615331d,NaN,4
120107,59862,59862,2e38823a-fbcb-4fc5-94bf-8457417c1d20,None,None,d48a254d-dd98-471e-a506-622f30774009,4971858637664481,WEB-1388012W,x6-9994287R,NaN,4
120108,66258,66257,ffcaae77-a983-41db-9ea1-65abb4036bc1,None,None,45447a05-97cf-42ac-b839-5f1416945e41,4971858637664481,HA-FC4E6AFF,c7-6364718G,NaN,1
120109,67926,67926,c9c2a9c7-3ef3-4c8b-8eab-01c5e531039e,None,None,4cac12fb-b8f2-46c5-874a-1df4f847c6a2,4971858637664481,VE-1050754E,k9-3755441k,NaN,5
120110,77538,77538,1f049481-3fa2-498b-8a60-66b60a71d376,Friedemann,Jüttner,2e5d948f-3a52-42c5-ab33-3a162208915b,4971858637664481,WEB-1388012W,o6-5976806E,NaN,2


In [7]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120123 entries, 0 to 120122
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   level_0           120123 non-null  int64  
 1   index             120123 non-null  int64  
 2   date_uuid         120123 non-null  object 
 3   first_name        15284 non-null   object 
 4   last_name         15284 non-null   object 
 5   user_uuid         120123 non-null  object 
 6   card_number       120123 non-null  int64  
 7   store_code        120123 non-null  object 
 8   product_code      120123 non-null  object 
 9   1                 0 non-null       float64
 10  product_quantity  120123 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 10.1+ MB


In [65]:
# drop first name, last name and 1
table.drop(['first_name','last_name','1'], axis =1, inplace = True)

In [67]:
table.drop(['level_0','index'], axis = 1, inplace= True)

In [68]:
table.head(5)

,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2


In [70]:
table['product_quantity'].value_counts()

product_quantity
3     22497
1     22440
4     22385
2     22157
6      5067
5      5064
8      4926
7      4864
12     1549
14     1544
15     1542
13     1536
11     1523
9      1522
10     1507
Name: count, dtype: int64

In [6]:
from data_cleaning import DataCleaning
cleaning = DataCleaning(table)
cleaned = cleaning.clean_orders_data()


In [7]:
print(cleaned.head(5))

                              date_uuid                             user_uuid  \
0  9476f17e-5d6a-4117-874d-9cdb38ca1fa6  93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8   
1  0423a395-a04d-4e4a-bd0f-d237cbd5a295  8fe96c3a-d62d-4eb5-b313-cf12d9126a49   
2  65187294-bb16-4519-adc0-787bbe423970  fc461df4-b919-48b2-909e-55c95a03fe6b   
3  579e21f7-13cb-436b-83ad-33687a4eb337  6104719f-ef14-4b09-bf04-fb0c4620acb0   
4  00ab86c3-2039-4674-b9c1-adbcbbf525bd  9523a6d3-b2dd-4670-a51a-36aebc89f579   

        card_number    store_code product_code  product_quantity  
0    30060773296197   BL-8387506C  R7-3126933h                 3  
1   349624180933183  WEB-1388012W  C2-7287916l                 2  
2  3529023891650490   CH-01D85C8D  S7-1175877v                 2  
3   213142929492281   CL-C183BE4B  D8-8421505n                 2  
4      502067329974   SO-B5B9CB3B  B6-2596063a                 2  


In [8]:
connect.upload_to_db(cleaned, 'orders_table')

### Extracting sales_date from S3

In [11]:
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

connect = DatabaseConnector()
connect.read_db_creds()
connect.init_db_engine()
connect.list_db_tables()

extract = DataExtractor()
table = extract.extract_from_s3("s3://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json")
print(table)

legacy_store_details
legacy_users
orders_table
Error: An error occurred (NoSuchBucket) when calling the GetObject operation: The specified bucket does not exist
None


In [1]:
# Initialize Boto 3 S3 client
import boto3

s3 = boto3.client('s3')
# Read the file from S3
try:
    response = s3.get_object(Bucket="https://data-handling-public.s3.eu-west-1.amazonaws.com", Key="date_details.json")
    # Access and read the content of the file. Data type = Bytes due to get request
    data = response['Body'].read()

     # Process the data as needed
    df = pd.read_csv(BytesIO(data), index_col = 0)  # Replace this with your processing logic
    print(df)
        
except Exception as e:
    print(f"Error: {e}")

Error: Parameter validation failed:
Invalid bucket name "https://data-handling-public.s3.eu-west-1.amazonaws.com": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"


In [2]:
import json
import requests
url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

KeyboardInterrupt: 

In [6]:
import pandas as pd
dict_df = pd.DataFrame(data)

In [7]:
print(dict_df)

       timestamp month  year day time_period  \
0       22:00:06     9  2012  19     Evening   
1       22:44:06     2  1997  10     Evening   
2       10:05:37     4  1994  15     Morning   
3       17:29:27    11  2001   6      Midday   
4       22:40:33    12  2015  31     Evening   
...          ...   ...   ...  ..         ...   
120156  22:56:56    11  2022  12     Evening   
120157  18:25:20     5  1997  31     Evening   
120158  18:21:40     9  2011  13     Evening   
120159  19:10:53     7  2013  12     Evening   
120160  21:17:12     3  2008  18     Evening   

                                   date_uuid  
0       3b7ca996-37f9-433f-b6d0-ce8391b615ad  
1       adc86836-6c35-49ca-bb0d-65b6507a00fa  
2       5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31  
3       1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8  
4       dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44  
...                                      ...  
120156  d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7  
120157  f7722027-1aae-49c3-8f8d-853e93f9f3e6  


In [11]:
# Concatenate the columns and convert to date
dict_df['date'] = pd.to_datetime(dict_df['year'].astype(str) + '-' + dict_df['month'].astype(str) + '-' + dict_df['day'].astype(str) + ' ' + dict_df['timestamp'], format = 'mixed', errors = 'coerce')

In [13]:
dict_df.drop(['day','month','year','timestamp'], axis = 1, inplace = True)

In [14]:
dict_df.head(5)

,time_period,date_uuid,date
0,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad,2012-09-19 22:00:06
1,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa,1997-02-10 22:44:06
2,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31,1994-04-15 10:05:37
3,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8,2001-11-06 17:29:27
4,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44,2015-12-31 22:40:33


In [29]:
null_df = dict_df.index[dict_df.isnull().any(axis=1)]

In [30]:
dict_df.loc[null_df]

,time_period,date_uuid,date


In [28]:
dict_df.drop(null_df, inplace = True)

In [1]:
from data_extraction import DataExtractor

extract = DataExtractor()
get_json = extract.extract_json_s3('https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json')

In [2]:
get_json.head(5)

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [2]:
from data_cleaning import DataCleaning

clean = DataCleaning(get_json)
cleaned = clean.clean_json_s3()
print(cleaned.head(5))

  time_period                             date_uuid                date
0     Evening  3b7ca996-37f9-433f-b6d0-ce8391b615ad 2012-09-19 22:00:06
1     Evening  adc86836-6c35-49ca-bb0d-65b6507a00fa 1997-02-10 22:44:06
2     Morning  5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31 1994-04-15 10:05:37
3      Midday  1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8 2001-11-06 17:29:27
4     Evening  dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44 2015-12-31 22:40:33


In [3]:
from database_utils import DatabaseConnector

connect = DatabaseConnector()
upload = connect.upload_to_db(cleaned,"dim_date_times")